## Note: mode1-taxi, mode2-FHV, mode3-shared FHV, mode4-PT, mode5-walking

In [21]:
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

## Data

In [22]:
acs = pd.read_csv('final_acs_transportation_choice.csv')#, index_col=0)
acs.head()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
0,3.0,0.228957,35.391246,14.547350,7706.507979,949.324468
1,4.0,46.244797,152.049702,91.310873,7487.249289,2840.145340
2,5.0,0.349401,56.498123,9.104389,7989.174863,173.873224
3,6.0,0.263186,18.439186,4.534672,4767.467108,478.295847
4,7.0,29.894066,167.041570,44.019826,33307.536619,3139.507920


In [23]:
mode_data = pd.read_csv('final_allMode_with_wage_cleaned.csv', index_col=0)
mode_data = mode_data.dropna()
mode_data.head()

,PUlocationID,DOlocationID,duration,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,mode,nest
ODpair,,,,,,,,,,,,,,,,,
3-4,3,4,39.695000,64.000000,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005,2,1
3-4,3,4,45.216667,61.500000,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005,3,1
3-7,3,7,47.880952,43.157143,1.504915,2.161294,3.890709,1.693267,2.488532,3.552056,4.027693,5.63154,3.521615,4.444350,3.084029,1,1
3-7,3,7,30.521739,47.000000,1.504915,2.161294,3.890709,1.693267,2.488532,3.552056,4.027693,5.63154,3.521615,4.444350,3.084029,2,1
3-7,3,7,37.159009,45.000000,1.504915,2.161294,3.890709,1.693267,2.488532,3.552056,4.027693,5.63154,3.521615,4.444350,3.084029,3,1


In [24]:
mode_data_2 = mode_data.copy()

In [25]:
# mode_data_2.reset_index(inplace=True)

In [26]:
mode_data_2.columns

Index(['PUlocationID', 'DOlocationID', 'duration', 'price', '2500', '7500',
       '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000',
       '225000', 'mode', 'nest'],
      dtype='object')

In [27]:
# mode_data_2['O'] = mode_data_2['ODpair'].apply(lambda x: str(x).split('-')[0]).astype('int')
# mode_data_2['D'] = mode_data_2['ODpair'].apply(lambda x: str(x).split('-')[1]).astype('int')

In [28]:
# mode_data_2.set_index('ODpair',inplace=True)

In [29]:
selected_zones = [140,141, 237, 236, 263, 262, 43, 238, 239, 143,142,12, \
                  88, 261, 13, 87, 209, 231, 45, 232, 148, 144, 211, 125, 158,\
                  249, 114, 113, 79, 4, 224, 107, 234, 90, 68, 246, 186, 164, 100,\
                  170, 137, 233, 162, 161, 230, 48, 50, 163, 229]

In [30]:
mode_data_2 = mode_data_2[(mode_data_2.PUlocationID.isin(selected_zones)) | (mode_data_2.DOlocationID.isin(selected_zones))]

In [31]:
mode_data_2.head()

,PUlocationID,DOlocationID,duration,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,mode,nest
ODpair,,,,,,,,,,,,,,,,,
3-4,3,4,39.695000,64.0,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.938590,0.586936,0.740725,0.514005,2,1
3-4,3,4,45.216667,61.5,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.938590,0.586936,0.740725,0.514005,3,1
3-13,3,13,54.867754,69.5,0.543441,0.780467,1.404978,0.611458,0.898637,1.282687,1.454445,2.033612,1.271694,1.604904,1.113677,2,1
3-13,3,13,38.500000,66.5,0.543441,0.780467,1.404978,0.611458,0.898637,1.282687,1.454445,2.033612,1.271694,1.604904,1.113677,3,1
3-43,3,43,41.079851,61.0,0.376229,0.540323,0.972677,0.423317,0.622133,0.888014,1.006923,1.407885,0.880404,1.111088,0.771007,2,1


In [32]:
mode_data_2.loc[mode_data_2['mode']==1,'price'] = mode_data_2.loc[mode_data_2['mode']==1,'price']+2.5
mode_data_2.loc[mode_data_2['mode']==2,'price'] = mode_data_2.loc[mode_data_2['mode']==2,'price']+2.75
mode_data_2.loc[mode_data_2['mode']==3,'price'] = mode_data_2.loc[mode_data_2['mode']==3,'price']+0.75

In [33]:
mode_data_2.head()

,PUlocationID,DOlocationID,duration,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,mode,nest
ODpair,,,,,,,,,,,,,,,,,
3-4,3,4,39.695000,66.75,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.938590,0.586936,0.740725,0.514005,2,1
3-4,3,4,45.216667,62.25,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.938590,0.586936,0.740725,0.514005,3,1
3-13,3,13,54.867754,72.25,0.543441,0.780467,1.404978,0.611458,0.898637,1.282687,1.454445,2.033612,1.271694,1.604904,1.113677,2,1
3-13,3,13,38.500000,67.25,0.543441,0.780467,1.404978,0.611458,0.898637,1.282687,1.454445,2.033612,1.271694,1.604904,1.113677,3,1
3-43,3,43,41.079851,63.75,0.376229,0.540323,0.972677,0.423317,0.622133,0.888014,1.006923,1.407885,0.880404,1.111088,0.771007,2,1


In [34]:
mode_data_2.shape

(89855, 17)

## Modeling

### Nested Logit Model

In [35]:
def utility(mode, wage, Lambda, dataset):
    '''
    Get the utility for mode j under specific OD pair
    Lambda: parameter that trade-off different transportation mode
    '''
    subset = dataset[dataset['mode'] == mode]
    vj = Lambda * (float(int(wage)/124800) * float(subset['duration']) + float(subset['price'])) #525600: convert wage scale 'year' to 'minitues'
    return -vj #assign negative utility?


def InclusiveValue(Nk, Tk, wage, dictVj, Lambda, dataset):
    '''
    Get the inclusive value for nest K
    Nk:nest k
    T: the dict that contains Tao(dissmilarity parameter) for each Nest. for example, Tk meeas Tao for nest K
    '''
    subsetNk = dataset[dataset['nest'] == Nk]
    modes = list(subsetNk['mode'].unique()) #what modes contained in this nest k
    sumIV = 0
    for j in modes:   
        vj = utility(j, wage, Lambda, subsetNk)
        dictVj[j] = vj
        sumIV += np.exp((1/Tk)*vj)  
    IVk = np.log(sumIV)
    return dictVj, IVk
    

def denoSum(T, nestList, wage, Lambda, dataset):
    '''
    Calculate the denomenator for P(y=Nk)
    T: the dict that contains Tk for each Nest, in our case T={1:T1, 2:T2, 3:T3}; T1, T2, T3 defined by us 
    TotK: the total number of nest this model has, in our case TotK = 3
    '''
    deno = 0
    dictIVk = {}
    dictVj = {}
    for Nk in nestList: #k is the k Nest, in our case k=1,2,3
        Tk = T[Nk] #get the tao for nest k 
        dictVj, IVk = InclusiveValue(Nk, Tk, wage, dictVj, Lambda, dataset)
        denok = np.exp(Tk*IVk)
        deno += denok
        dictIVk[Nk] = IVk
    return dictVj, dictIVk, deno


def probability (j, Nk, T, dictVj, dictIVk, deno):  
    '''
    Calculate the probability for the specific mode j and Nest Nk
    '''
    Tk = T[Nk]
    IVk = dictIVk[Nk]
    vj = dictVj[j]
    pjk = np.exp((1/Tk)*vj)/np.exp(IVk) #pjk: P(y=j, y belong to Nk)
    pk = np.exp(Tk*IVk)/deno #pk: P(y belong to Nk)
    pj = pjk*pk #pj: P(y=j)
    return pj

### Apply to our case

In [36]:
def apply_model(T, nestList, wageList, Lambda, dataAll):
    '''
    apply to our case
    '''
    import timeit
    start = timeit.default_timer()

    ODpair_list = list(dataAll.index.unique())
    p = []
    for i in tqdm(ODpair_list,position=0): #indentify OD pair  
        dataOD = dataAll[dataAll.index==i] 
        modeList = list(dataOD['mode'])
        nestList = list(dataOD['nest'])
        
        pop_OD = [] #store the results under each OD pair
        for wage in wageList:            
            dictVj, dictIVk, deno = denoSum(T, set(nestList), wage, Lambda, dataOD)
            pop_mode = [] #store the results under each OD pair and each wage
            for i in range(1,6):
                if i in modeList: #not all modes appear in every OD pair
                    pop = dataOD[wage].mean() * probability(i, nestList[modeList.index(i)], T, dictVj, dictIVk, deno)
#                     print(wage, i,nestList[modeList.index(i)], pop)
                    if np.isnan(pop) == True: #if predicted population is nan, replace it as 0, means no people choose
                        pop = 0
                
                    pop_mode.append(pop)
                else: 
                    pop_mode.append(0)
            pop_OD.append(pop_mode)
        pop_OD_sum = [sum(x) for x in zip(*pop_OD)] #sum the weighted population under each wage scenario
        p.append(pop_OD_sum)

    df = pd.DataFrame(p, columns=['P(mode1)', 'P(mode2)', 'P(mode3)', 'P(mode4)', 'P(mode5)']
                      , index=ODpair_list)
    
    stop = timeit.default_timer()
    timeslot = stop - start
    return df, timeslot

def compare_with_ground_truth(predictdf, truedf):
    '''
    compare our predicted transportation choice with ground truth
    
    The header of the datafrme after merge (named 'data_compare') should be like:
    taxi_zone | P(mode1)_x | P(mode2)_x | P(mode3)_x | P(mode4)_x | P(mode5)_x | P(mode1)_y | P(mode2)_y | P(mode3)_y | P(mode4)_y | P(mode5)_y
    '''
    import numpy as np
    # makesure predictdf and truedf have the same formats
    for col in predictdf.columns:
        predictdf[col] = predictdf[col].astype(float) 
    predictdf = predictdf.fillna(0)
    predictdf = predictdf.replace([np.inf, -np.inf], np.nan)
    predictdf = predictdf.dropna()
    predictdf['taxi_zone'] = predictdf.index.map(lambda x: x.split('-')[0]) #get origin taxi zone from each OD pair
    predictdf = predictdf.groupby('taxi_zone').sum().reset_index() #group the popuation by taxi zone
    predictdf['taxi_zone'] = predictdf['taxi_zone'].astype(int)
    truedf['taxi_zone'] = truedf['taxi_zone'].astype(int)
    
    data_compare = pd.merge(predictdf, truedf, left_on='taxi_zone', right_on = 'taxi_zone', how = 'left')
    data_compare = data_compare.dropna() 
    
    rmse = 0
    for i in range(1,6):
        rmsei = np.sqrt(sum((data_compare[data_compare.columns[i]] - data_compare[data_compare.columns[i+5]])**2))
    rmse += rmsei
    return data_compare, rmse

In [37]:
#choose ODpair 3-1 to test the algrithm whether bug-free:
T1 = 10
T2 = 1
T3 = 1
T = {1:T1, 2:T2, 3:T3} #Tao for each nest
Lambda = 2 
nestList = [1, 2, 3]
wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']
testdf = mode_data[mode_data.index.isin(list(mode_data.index[:10]))] #test OD pair 3-1,3-2,3-3,3-4

predict_choice_test, timeslot_test = apply_model(T, nestList, wagelist, Lambda, testdf)
results_test,rmse_test = compare_with_ground_truth(predict_choice_test, acs)
print('The time used to run the code:', timeslot_test)
print('The rmse of this model is:', rmse_test)
print()
print('The predict transportation choice is:')
predict_choice_test

100%|██████████| 5/5 [00:00<00:00, 14.57it/s]


The time used to run the code: 0.3477434739470482
The rmse of this model is: 947.227430122

The predict transportation choice is:


,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
3-4,0.000000,4.195442e-01,0.094461,5.235710,0.250285
3-7,0.032738,3.028392e+00,0.496368,30.953772,1.488730
3-9,0.000000,5.531724e-01,0.067798,1.301644,0.077386
3-10,0.000000,6.274876e-01,0.000183,2.247016,0.125314
3-11,0.000000,2.237352e-64,0.000000,3.844677,0.155323


In [38]:
#Run the model for the whole dataset
T1 = 0.1 #should set grid search for T1 
T2 = 1
T3 = 1
T = {1:T1, 2:T2, 3:T3} #Tao for each nest
Lambda = 0.01 #should set grid search for Lambda 
nestList = [1, 2, 3]
wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']

predict_transportation_choice, timeslot = apply_model(T, nestList, wagelist, Lambda, mode_data_2)
results,rmse = compare_with_ground_truth(predict_transportation_choice, acs)

100%|██████████| 19384/19384 [35:14<00:00,  9.17it/s]


In [39]:
rmse

16461.20395169776

In [40]:
predict_transportation_choice.to_csv('results_2.csv')

In [71]:
rmse

22663.084021505307

In [72]:
predict_transportation_choice.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
3-3,273.000000,0.000000,0.000000,0.000000,0.000000
3-4,0.000000,0.686666,0.519443,3.842061,0.951830
3-7,2.307236,5.386499,3.704195,19.934948,4.667122
3-9,0.000000,0.616366,0.301406,0.950454,0.131774
3-10,0.000000,0.853353,0.100890,1.810278,0.235479


In [73]:
predict_transportation_choice.to_csv('Results_scenario2.csv')

In [37]:
#Run the model for the whole dataset
T1 = 0.5 #should set grid search for T1 
T2 = 1
T3 = 1
T = {1:T1, 2:T2, 3:T3} #Tao for each nest
Lambda = 22 #should set grid search for Lambda 
nestList = [1, 2, 3]
wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']

predict_transportation_choice, timeslot = apply_model(T, nestList, wagelist, Lambda, mode_data[:7242])
results,rmse = compare_with_ground_truth(predict_transportation_choice, acs[:7])

100%|██████████| 1742/1742 [01:11<00:00, 22.73it/s]


In [38]:
rmse

2058.6316307001193

In [ ]:
#Run the model for the whole dataset
T1 = 3000 #should set grid search for T1 
T2 = 1
T3 = 1
T = {1:T1, 2:T2, 3:T3} #Tao for each nest
Lambda = 2 #should set grid search for Lambda 
nestList = [1, 2, 3]
wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']

predict_transportation_choice, timeslot = apply_model(T, nestList, wagelist, Lambda, mode_data)
combine, rmse = compare_with_ground_truth(predict_transportation_choice, acs)

In [9]:
print('The time used to run the code:', timeslot)
print('The mse of this model is:', mse)
print()
print('The predict transportation choice is:')
predict_transportation_choice

The time used to run the code: 3545.58184884
The mse of this model is: 4004786.8137141904

The predict transportation choice is:


,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
3-1,561.083498,5.583506e+02,501.565913,0.000000,0.000000e+00
3-2,0.000000,0.000000e+00,0.000000,6170.000000,1.164735e-43
3-3,18922.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
3-4,0.000000,8.481136e+02,772.886437,0.000000,0.000000e+00
3-5,0.000000,0.000000e+00,0.000000,4862.000000,1.407611e-83
3-6,0.000000,6.181992e+03,0.000000,3972.008201,2.108063e-54
3-7,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
3-8,0.000000,0.000000e+00,0.000000,8037.000000,1.894266e-28
3-9,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
3-10,0.000000,2.475536e+03,1481.464104,0.000000,0.000000e+00
